In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
from collections import defaultdict
from joblib import Parallel, delayed
import dill

from Metrics import rpf1
from load_data import load_process_essays
from wordtagginghelper import merge_dictionaries

#from gensim.models import Word2Vec
from window_based_tagger_config import get_config
from IdGenerator import IdGenerator as idGen
from results_procesor import ResultsProcessor, __MICRO_F1__
from Rpfa import micro_rpfa
from collections import defaultdict

import Settings
import logging
import datetime

In [3]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from load_data import load_process_essays
from collections import defaultdict
from IterableFP import flatten
from Settings import Settings
from Settings import Settings

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
root_folder = settings.data_directory + "CoralBleaching/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
training_pickled = settings.data_directory + "CoralBleaching/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)
processor = ResultsProcessor()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [4]:
with open(training_pickled, "rb+") as f:
    tagged_essays = pickle.load(f)
len(tagged_essays)

902

In [5]:
import datetime, logging
print("Started at: " + str(datetime.datetime.now()))
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
logger = logging.getLogger()

Started at: 2017-07-03 12:20:56.649966


In [6]:
tag_freq = defaultdict(int)
unique_words = set()
for essay in tagged_essays:
    for sentence in essay.sentences:
        for word, tags in sentence:
            unique_words.add(word)
            for tag in tags:
                tag_freq[tag] += 1

EMPTY_TAG = "Empty"
#TODO - don't ignore Anaphor, other and rhetoricals here
cr_tags = list((t for t in tag_freq.keys() if ( "->" in t) and not "Anaphor" in t and not "other" in t and not "rhetorical" in t))
regular_tags = list((t for t in tag_freq.keys() if t[0].isdigit()))

vtags = set(regular_tags)
#vtags.add(EMPTY_TAG)
#cr_tags = vtags

len(unique_words)

1641

In [7]:
regular_tags

['50', '5', '4', '11', '13', '3', '7', '1', '6', '14', '2', '12', '5b']

In [8]:
cr_tags

['Causer:5->Result:50',
 'Causer:7->Result:50',
 'Causer:3->Result:4',
 'Causer:1->Result:50',
 'Causer:13->Result:50',
 'Causer:11->Result:50',
 'Causer:6->Result:50',
 'Causer:3->Result:5',
 'Causer:4->Result:14',
 'Causer:3->Result:1',
 'Causer:1->Result:3',
 'Causer:3->Result:50',
 'Causer:4->Result:5',
 'Causer:3->Result:7',
 'Causer:6->Result:7',
 'Causer:11->Result:12',
 'Causer:12->Result:13',
 'Causer:7->Result:5b',
 'Causer:5b->Result:50',
 'Causer:50->Result:50',
 'Causer:1->Result:2',
 'Causer:2->Result:3',
 'Causer:11->Result:3',
 'Causer:11->Result:13',
 'Causer:11->Result:14',
 'Causer:5->Result:5b',
 'Causer:1->Result:7',
 'Causer:7->Result:14',
 'Causer:5->Result:7',
 'Causer:11->Result:6',
 'Causer:13->Result:14',
 'Causer:3->Result:13',
 'Causer:3->Result:14',
 'Causer:4->Result:50',
 'Causer:14->Result:50',
 'Causer:1->Result:14',
 'Causer:4->Result:5b',
 'Causer:1->Result:4',
 'Causer:6->Result:14',
 'Causer:5b->Result:7',
 'Causer:3->Result:5b',
 'Causer:13->Resul

### Do CRE Tags Overlap? Yes......

In [9]:
CAUSER = "Causer"
RESULT = "Result"
EXPLICIT = "explicit"

In [10]:
cre = set([CAUSER, EXPLICIT, RESULT])
cnt, total_wds = 0,0
total_cr_s, total_ovr_cr_s = 0,0
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        prev_itags = set()
        un_cr_tags = set()
        un_ovr_cr_tags = set()
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1
            itags = cre.intersection(tags)
            crt = tags.intersection(cr_tags)
            un_cr_tags.update(crt)
            if len(itags) > 1:
                new_ones = []
                for t in itags:
                    if t not in prev_itags:
                        new_ones.append(t)
                if len(new_ones) > 1:  
                    print((essay.name, eix,six,wix, str(new_ones), len(itags), ",".join(tags)))
                    cnt+=1
            if len(crt) > 1:
                un_ovr_cr_tags.update(crt)
            prev_itags = itags
        total_cr_s += len(un_cr_tags)
        total_ovr_cr_s += len(un_ovr_cr_tags)

('EBA1415_BGJD_2_CB_ES-05736.ann', 12, 9, 8, "['Result', 'explicit']", 2, 'Result,14,explicit,Causer:4->Result:14,Result:14')
('EBA1415_BGJD_2_CB_ES-05751.ann', 17, 5, 7, "['Result', 'explicit']", 2, 'Result,14,explicit,Causer:4->Result:14,Result:14')
('EBA1415_BGJD_2_CB_ES-05756.ann', 21, 1, 8, "['Result', 'Causer']", 2, 'Result,Causer:3->Result:7,Causer,Causer:7->Result:50,7,Result:7,Causer:7')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 3, 6, "['Result', 'explicit']", 3, 'Result,Causer:12,explicit,Causer,Causer:11->Result:12,Causer:12->Result:13,12,Result:12')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 3, 19, "['Result', 'Causer']", 2, 'Result:13,Result,Causer,Causer:13,Causer:12->Result:13,Causer:13->Result:50,13')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 8, 23, "['Result', 'Causer']", 2, 'Result,5b,Result:5b,Causer,Causer:5b->Result:50,Causer:5b,Causer:7->Result:5b')
('EBA1415_BLHT_5_CB_ES-05204.ann', 24, 13, 8, "['Result', 'Causer']", 2, 'Causer:1->Result:50,Result,Result:50,Causer,Causer:5

('EBA1415_KYLS_6_CB_ES-05690.ann', 188, 2, 13, "['Result', 'Causer']", 2, 'Result,Causer:6->Result:14,14,Causer:14->Result:50,Causer:14,Causer,Result:14')
('EBA1415_KYLS_6_CB_ES-05691.ann', 189, 2, 7, "['Result', 'Causer']", 2, 'Result,Causer:6->Result:7,Causer,Causer:7->Result:50,7,Result:7,Causer:7')
('EBA1415_KYLS_6_CB_ES-05691.ann', 189, 2, 11, "['Result', 'Causer']", 2, 'Result,Causer:6->Result:7,Causer,Causer:7->Result:50,7,Result:7,Causer:7')
('EBA1415_KYLS_6_CB_ES-05691.ann', 189, 2, 15, "['Result', 'explicit']", 2, 'Result,Result:50,explicit,50,Causer:7->Result:50')
('EBA1415_KYLS_6_CB_ES-05691.ann', 189, 4, 11, "['Result', 'Causer']", 2, 'Result,Result:50,Causer,Causer:50,50,Causer:13->Result:50,Causer:50->Result:50')
('EBA1415_KYLS_6_CB_ES-05691.ann', 189, 5, 12, "['Result', 'Causer']", 2, 'Result,Causer:12,Causer,Causer:11->Result:12,Causer:12->Result:13,12,Result:12')
('EBA1415_KYNS_3_CB_ES-05371.ann', 190, 3, 16, "['explicit', 'Causer']", 2, 'Causer:6,explicit,Causer,Caus

('EBA1415_SEKL_23_CB_ES-04823.ann', 427, 3, 10, "['Result', 'Causer']", 2, 'Result:13,Result,Causer,Causer:13,Causer:13->Result:14,Causer:11->Result:13,13')
('EBA1415_SEKL_23_CB_ES-04828.ann', 429, 4, 3, "['Result', 'explicit']", 2, 'Result,Result:2,Causer:1->Result:2,explicit')
('EBA1415_SEKL_2_CB-04829.ann', 432, 4, 6, "['Result', 'Causer']", 2, 'Result,Causer:3->Result:7,Result:3,Causer,Causer:1->Result:3,3,Causer:3')
('EBA1415_SEKL_2_CB-04829.ann', 432, 6, 1, "['Result', 'explicit']", 2, 'Result,Causer:Anaphor->Result:4,4,explicit,Result:4')
('EBA1415_SEKL_2_CB-04829.ann', 432, 7, 6, "['Result', 'explicit']", 2, 'Result,5,Causer:4->Result:5,explicit,Result:5')
('EBA1415_SEKL_2_CB-04829.ann', 432, 8, 3, "['Result', 'Causer']", 2, 'Result,Causer:Anaphor->Result:7,Causer,Causer:7->Result:50,7,Result:7,Causer:7')
('EBA1415_SERS_1314_CB-05097.ann', 435, 7, 2, "['Result', 'explicit']", 3, 'Result,Causer:12,explicit,Causer,Causer:11->Result:12,Causer:12->Result:13,12,Result:12')
('EBA1415

('EBA1415_TTCM_910_CB_ES-04930.ann', 697, 0, 1, "['Result', 'Causer']", 2, 'Result,Causer:6->Result:7,Causer,Causer:7->Result:50,7,Result:7,Causer:7')
('EBA1415_TTCM_910_CB_ES-05974.ann', 699, 0, 17, "['Result', 'Causer']", 2, 'Result,Causer:6->Result:14,14,Causer:14->Result:50,Causer:14,Causer,Result:14')
('EBA1415_TTCM_910_CB_ES-06152.ann', 704, 2, 3, "['Result', 'explicit']", 3, 'Result,Causer:12,explicit,Causer,Causer:11->Result:12,Causer:12->Result:13,12,Result:12')
('EBA1415_TTCM_910_CB_ES-06154.ann', 705, 10, 1, "['Result', 'explicit']", 2, 'Result:6,Result,Causer:Anaphor->Result:6,explicit,6')
('EBA1415_TTCM_910_CB_ES-06156.ann', 707, 1, 22, "['Result', 'Causer']", 2, 'Result,Result:3,Causer,Causer:1->Result:3,Causer:3->Result:4,3,Causer:3')
('EBA1415_TTCM_910_CB_ES-06156.ann', 707, 1, 26, "['Result', 'Causer']", 2, 'Result,Result:3,Causer,Causer:1->Result:3,Causer:3->Result:4,3,Causer:3')
('EBA1415_TTCM_910_CB_ES-06156.ann', 707, 2, 1, "['Result', 'explicit']", 2, 'Result,14,e

In [11]:
# how many of the instances of Causal tags are overlapping?
total_cr_s, total_ovr_cr_s # over 1/3

(3006, 1163)

In [12]:
cnt, total_wds

(559, 137166)

In [13]:
cre = set([CAUSER, EXPLICIT, RESULT])
cnt, total_wds = 0,0
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        prev_itags = set()
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1            
            for t in tags:
                if "->" in t:
                    itags = cre.intersection(tags)
                    if len(itags) <=0:
                        print((essay.name, eix,six,wix, len(itags), ",".join(tags)))
            prev_itags = itags

('EBA1415_AEKD_4_CB_ES-05572.ann', 1, 1, 8, 0, 'Causer:5->Result:50')
('EBA1415_AEKD_4_CB_ES-05572.ann', 1, 1, 10, 0, 'Causer:5->Result:50')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 0, 21, 0, 'Causer:7->Result:50')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 0, 23, 0, 'Causer:7->Result:50')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 5, 0, 'Causer:3->Result:4')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 6, 0, 'Causer:3->Result:4')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 7, 0, 'Causer:3->Result:4')
('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 6, 7, 0, 'Causer:7->Result:50')
('EBA1415_AEKD_5_CB_ES-05582.ann', 4, 1, 7, 0, 'Causer:7->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 7, 0, 'Causer:1->Result:50,Causer:11->Result:50,Causer:13->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 7, 0, 'Causer:1->Result:50,Causer:11->Result:50,Causer:13->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 7, 0, 'Causer:1->Result:50,Causer:11->Result:50,Causer:13->Result:50')
('EBA1415_BGJD_1_CB_ES-05727.ann',

('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 0, 20, 0, 'Causer:11->Result:50')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 0, 22, 0, 'Causer:11->Result:50')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 0, 23, 0, 'Causer:11->Result:50')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 0, 24, 0, 'Causer:11->Result:50')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 0, 26, 0, 'Causer:11->Result:50')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 1, 27, 0, 'Causer:1->Result:3')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 2, 1, 0, 'Causer:other->Result:50')
('EBA1415_KNKC_3_CB_ES-05601.ann', 149, 2, 8, 0, 'Causer:other->Result:3')
('EBA1415_KNKC_3_CB_ES-05603.ann', 150, 1, 1, 0, 'Causer:1->Result:Anaphor')
('EBA1415_KNKC_3_CB_ES-05603.ann', 150, 1, 2, 0, 'Causer:1->Result:Anaphor')
('EBA1415_KNKC_3_CB_ES-05603.ann', 150, 1, 3, 0, 'Causer:1->Result:Anaphor')
('EBA1415_KNKC_3_CB_ES-05603.ann', 150, 1, 5, 0, 'Causer:1->Result:Anaphor')
('EBA1415_KNKC_3_CB_ES-05603.ann', 150, 1, 6, 0, 'Causer:1->Result:Anaphor')
('EBA1415_KNKC_3_CB_ES

('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 16, 32, 0, 'Causer:5->Result:50')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 7, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 8, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 9, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 10, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 11, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 12, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 13, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 14, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 15, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 16, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 17, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411, 17, 19, 0, 'Causer:6->Result:7')
('EBA1415_SEAL_78_CB_ES-04803.ann', 411,

('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 2, 24, 0, 'Causer:3->Result:5')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 2, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 3, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 4, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 6, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 7, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 8, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 9, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 10, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 4, 11, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05363.ann', 813, 6, 11, 0, 'Causer:11->Result:13')
('EBA1415_WSAL_2_CB_ES-05366.ann', 814, 0, 4, 0, 'Causer:rhetorical->Result:50')
('EBA1415_WSAL_2_CB_ES-05366.ann', 814, 0, 5, 0, 'Causer:rhetorical->Result:50')
('EBA1415_WSAL_2_CB_ES-05366.ann'

#### Any C->E Sequence Lack all 3 Sub-tags?

In [14]:
from collections import defaultdict

cre = set([CAUSER, EXPLICIT, RESULT])
cnt, total_wds = 0,0
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        cr_words = defaultdict(list)
        cr_tags = defaultdict(list)
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1            
            for t in tags:
                if "->" in t:
                    cr_words[t].append(word)
                    itags = cre.intersection(tags)
                    cr_tags[t].append(itags)
        
        for tag, lst in cr_tags.items():
            utags = set()
            for tgs in lst:
                utags.update(tgs)
            if len(utags) < 3:
                print((essay.name, eix, six, tag, len(utags), ",".join(utags)))

('EBA1415_AEKD_4_CB_ES-05902.ann', 2, 5, 'Causer:3->Result:4', 2, 'Causer,Result')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 'Causer:11->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_1_CB_ES-05727.ann', 7, 0, 'Causer:13->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05740.ann', 14, 0, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05740.ann', 14, 1, 'Causer:1->Result:3', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05751.ann', 17, 6, 'Causer:4->Result:5', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05752.ann', 18, 1, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05752.ann', 18, 3, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_BGJD_2_CB_ES-05756.ann', 21, 1, 'Causer:3->Result:7', 2, 'Causer,Result')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 4, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_BLHT_5_CB_ES-05202.ann', 23, 8, 'Causer:7->Result:5b', 2, 'Ca

('EBA1415_SEKL_1_CB-04816.ann', 416, 0, 'Causer:3->Result:4', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB-04816.ann', 416, 3, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB-04816.ann', 416, 3, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB-04817.ann', 417, 1, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB_ES-04815.ann', 423, 6, 'Causer:4->Result:5', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB_ES-04818.ann', 424, 1, 'Causer:4->Result:5', 2, 'Causer,Result')
('EBA1415_SEKL_1_CB_ES-04819.ann', 425, 11, 'Causer:11->Result:13', 2, 'Causer,Result')
('EBA1415_SEKL_23_CB_ES-04823.ann', 427, 1, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_23_CB_ES-04823.ann', 427, 2, 'Causer:1->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_23_CB_ES-04823.ann', 427, 2, 'Causer:1->Result:4', 2, 'Causer,Result')
('EBA1415_SEKL_23_CB_ES-04823.ann', 427, 2, 'Causer:3->Result:50', 2, 'Causer,Result')
('EBA1415_SEKL_23_CB_ES-04823.ann', 427, 2, 'Causer:3->Result:4'

### Are there Words with Multiple CR Tags? Yes also...

In [15]:
from collections import defaultdict

cnt, total_wds, total_sents, sents_with_cr = 0,0,0,0
sents_wds = defaultdict(list)
for eix, essay in enumerate(tagged_essays):
    for six, sentence in enumerate(essay.sentences):
        total_sents += 1
        has_cr_tag = False
        for wix, (word, tags) in enumerate(sentence):
            total_wds += 1
            crel_tags = set()
            for t in tags:
                if "->" in t:
                    has_cr_tag = True
                    crel_tags.add(t)
            if len(crel_tags) > 1:            
                sents_wds[(eix, six)].append(wix)
                #print((essay.name, eix, six, wix, tags,  crel_tags))
        if has_cr_tag:
            sents_with_cr += 1

In [16]:
len(sents_wds), total_sents, sents_with_cr

(597, 8292, 2611)

In [17]:
for (eix, six), wds in list(sents_wds.items())[:10]:
    essay = tagged_essays[eix]
    print(essay.name)
    print()
    sentence = essay.sentences[six]
    for wd, tags in sentence:
        csr_tags = ",".join(sorted([t for t in tags if "->" not in t and "Causer:" in t]))
        res_tags = ",".join(sorted([t for t in tags if "->" not in t and "Result:" in t]))
        cr_tags = ",".join(sorted([t.replace("Causer:","").replace("Result:","") for t in tags if "->" in t]))
        reg_tags = ",".join(sorted([t for t in tags if "->" not in t and (t[0].isdigit() or t == EXPLICIT)]))
        print(wd.ljust(15), reg_tags.ljust(15), cr_tags.ljust(30), csr_tags.ljust(20), res_tags)
    print("*" * 100)

EBA1415_BGJD_1_CB_ES-05727.ann

the                                                                                 
things                                                                              
that                                                                                
lead            explicit                                                            
to              explicit                                                            
coral           50              1->50,11->50,13->50                                 Result:50
bleaching       50              1->50,11->50,13->50                                 Result:50
could                           1->50,11->50,13->50                                 
be                              1->50,11->50,13->50                                 
salt            13              1->50,11->50,13->50            Causer:13            
that            13              1->50,11->50,13->50            Causer:13            
near           

In [18]:
def sort_key(crel):
    l,r = t.replace("b","").split("->")
    return (int(l), int(r))

for (eix, six), wds in list(sents_wds.items()):
    essay = tagged_essays[eix]
    sentence = essay.sentences[six]
    
    crel_tags = set()
    for wds, tags in sentence:
        for t in tags:
            if "->" in t:
                crel_tags.add(t.replace("Causer:","").replace("Result:",""))
    if len(crel_tags) <= 1:
        continue

    all_same_result = True
    result = None

    for t in crel_tags:
        l,r = t.split("->")
        if result is None:
            result = r
        else:
            if r != result:
                all_same_result = False
                break
    if all_same_result:
        #print(crel_tags)
        continue
    
    pairs = []
    for t in crel_tags:
        l,r = t.split("->")
        pairs.append((l, r))
    
    chain = [pairs[0]]
    pairs = pairs[1:]
    while len(pairs) > 0:
        found = False
        for ix, (l,r) in enumerate(list(pairs)):
            #print(l, chain[-1][1], chain[-1][1] ==l, r, chain[0][0], chain[0][0]==r)
            if l == chain[-1][-1]:
                chain.append((l,r))
                pairs = pairs[:ix] + pairs[ix+1:]
                found = True
                break
            if r == chain[0][0]:
                chain.insert(0, (l,r))
                pairs = pairs[:ix] + pairs[ix+1:]
                found = True
                break
        
        if not found:
            print(crel_tags)
            #print(chain)
            #print(pairs)
            #print()
            break
        else:
            pass

{'11->13', '11->3', '11->50'}
{'3->50', '1->50', '1->7', '3->7'}
{'7->14', '7->50'}
{'11->13', '11->14'}
{'5b->50', '5->50', '3->5'}
{'1->3', '1->50'}
{'Anaphor->3', 'Anaphor->50'}
{'11->13', '13->14', '12->13', '11->12'}
{'11->13', '12->13', '11->12'}
{'3->5', 'Anaphor->rhetorical', '5->rhetorical', 'Anaphor->3'}
{'Anaphor->14', 'Anaphor->13'}
{'1->3', '1->rhetorical'}
{'4->14', '1->3', '3->4', 'Anaphor->50'}
{'1->3', '1->Anaphor'}
{'11->rhetorical', '11->12'}
{'12->50', '11->6', '11->12'}
{'14->7', '14->50'}
{'4->13', '5->5b', '3->4'}
{'3->6', '3->50'}
{'other->6', 'other->50'}
{'11->13', '11->12'}
{'14->50', '7->50', '6->14'}
{'1->Anaphor', '1->13'}
{'6->7', '6->50'}
{'4->5', '3->50', '3->4'}
{'3->5b', '3->4'}
{'Anaphor->14', 'Anaphor->13'}
{'11->13', '12->13', '11->12'}
{'1->3', '1->50', '11->13', '13->50'}
{'1->other', '1->50', 'other->50'}
{'14->50', '13->14', '13->50'}
{'1->3', '1->50', '1->7'}
{'12->5b', '3->6', '11->12'}
{'13->14', '13->Anaphor'}
{'Anaphor->14', 'Anaphor->50'}

## How Many Causal Relations Lack the Causer or Effect (or both)?

In [23]:
def normalize_cr(cr):
    return cr.replace("Causer:","").replace("Result:","")

lr_tags = set()
for crel in cr_tags:
    l,r = crel.split("->")
    lr_tags.add(l)
    lr_tags.add(r)
    l,r = normalize_cr(crel).split("->")
    lr_tags.add(l)
    lr_tags.add(r)
sorted(lr_tags)

['1',
 '11',
 '12',
 '13',
 '14',
 '2',
 '3',
 '4',
 '5',
 '50',
 '5b',
 '6',
 '7',
 'Causer:1',
 'Causer:11',
 'Causer:12',
 'Causer:13',
 'Causer:14',
 'Causer:2',
 'Causer:3',
 'Causer:4',
 'Causer:5',
 'Causer:50',
 'Causer:5b',
 'Causer:6',
 'Causer:7',
 'Result:1',
 'Result:11',
 'Result:12',
 'Result:13',
 'Result:14',
 'Result:2',
 'Result:3',
 'Result:4',
 'Result:5',
 'Result:50',
 'Result:5b',
 'Result:6',
 'Result:7']

In [25]:
len(lr_tags), len(cr_tags)

(39, 82)

In [32]:
codes_per_sent = []
num_sent = 0
for essay_ix, essay in enumerate(tagged_essays):
    prev_sent = None
    for sent_ix, taggged_sentence in enumerate(essay.sentences):
        un_cr = set()
        un_reg = set()
        num_sent += 1
        for i, (wd,tags) in enumerate(taggged_sentence):
            reg_tags = tags.intersection(lr_tags)
            un_reg.update(reg_tags)
            crt = tags.intersection(cr_tags)
            un_cr.update(crt)
            current_word = wd
            
        if len(un_cr) > 0:
            codes_per_sent.append((un_cr,un_reg))

        for csl in un_cr:
            l,r = normalize_cr(csl).split("->")
            l1,r1 = csl.split("->")
            if l in un_reg and r in un_reg:
                continue
            if l1 in un_reg and r1 in un_reg:
                continue
            
            non_causal_prev = set()
            if prev_sent is not None:
                for wd,tags in prev_sent:
                    non_causal_prev.update([t for t in tags if "->" not in t])
            if l not in un_reg:
                print(l, csl, un_reg)
                print("Previous:",",".join(non_causal_prev))
                for i, (wd,tags) in enumerate(taggged_sentence):
                    non_causal = [t for t in tags if "->" not in t]
                    print(wd.ljust(30), ",".join(non_causal).ljust(20), "\t", csl if csl in tags else "")
                print()
                break
            elif r not in un_reg:
                print(r, csl, un_reg)
                print("Previous:",",".join(non_causal_prev))
                for i, (wd,tags) in enumerate(taggged_sentence):
                    non_causal = [t for t in tags if "->" not in t]
                    print(wd.ljust(30), ",".join(non_causal).ljust(20), "\t", csl if csl in tags else "")
                print()
                break
        prev_sent = taggged_sentence

1 Causer:1->Result:50 {'50', 'Result:50'}
Previous: 
there                                               	 
are                                                 	 
many                                                	 
factors                                             	 
that                                                	 
can                                                 	 
cause                                               	 
coral                          Result,Result:50,50  	 Causer:1->Result:50
bleaching                      Result,Result:50,50  	 Causer:1->Result:50
.                                                   	 

50 Causer:1->Result:50 {'Causer:6', '5', '1', 'Causer:5', 'Causer:1', '6'}
Previous: Result,50,Result:50
some                                                	 Causer:1->Result:50
happen                                              	 Causer:1->Result:50
naturally                                           	 Causer:1->Result:50
like                           

will                           Result,14,Result:14  	 Causer:13->Result:14
make                           Result,14,Result:14  	 Causer:13->Result:14
it                             Result,14,Result:14  	 Causer:13->Result:14
unhealthy                      Result,14,Result:14  	 Causer:13->Result:14
for                            Result,14,Result:14  	 Causer:13->Result:14
the                            Result,14,Result:14  	 Causer:13->Result:14
coral                          Result,14,Result:14  	 Causer:13->Result:14
.                                                   	 

50 Causer:6->Result:50 {'Causer:6', '6'}
Previous: 
now                                                 	 
,                                                   	 
like                                                	 
"                                                   	 
coral                                               	 
and                                                 	 
zooxanthellae                        

colors                         Result,50,Result:50  	 Causer:7->Result:50
,                                                   	 
and                                                 	 
turn                           Result,50,Result:50  	 Causer:7->Result:50
white                          Result,50,Result:50  	 Causer:7->Result:50
.                                                   	 

7 Causer:7->Result:50 {'Result:50', '50'}
Previous: 
many                                                	 
INFREQUENT                                          	 
the                                                 	 
death                                               	 
by                                                  	 
bleaching                      Result,50,Result:50  	 Causer:7->Result:50
.                                                   	 

50 Causer:7->Result:50 {'7', 'Causer:7'}
Previous: Result:50,Result,50
which                                               	 Causer:7->Result:50
can      

diseases                                            	 
and                                                 	 
starvation                     COMPILED             	 
.                                                   	 

3 Causer:3->Result:5 {'Causer:3'}
Previous: 3
according                                           	 
to                                                  	 
the                                                 	 
article                                             	 
,                                                   	 
reef                                                	 
building                                            	 
corals                                              	 
need                           Causer:3,Causer      	 Causer:3->Result:5
water                          Causer:3,Causer      	 Causer:3->Result:5
temperatures                   Causer:3,Causer      	 Causer:3->Result:5
INFREQUENT                     Causer:3,Causer      	 Causer:3->Result:5
.

## <span style="color:red">How to Solve for Sentences with Same Cause, Multiple Effects?</span>
* Dependency tree requires parent child relationship
* One cause, multiple effects means 1 child with multiple parents
* **Solution** - maybe link effects to effects in this scenario?

### Look For Non Projective Relations

In [19]:
for (eix, six), wds in list(sents_wds.items())[:10]:
    essay = tagged_essays[eix]
    sentence = essay.sentences[six]
    un_tags = []
    seen = set()
    c_rels = set()
    for wd, tags in sentence:
        #print(reg_tags, tags)
        for t in tags:
            if t not in seen:
                seen.add(t)
                if "->" not in t and (t[0].isdigit() or "Anaphor" == t or "rhetorical" == t):
                    un_tags.append(t)
                elif "->" in t:
                    c_rels.add(t)
    positions = []
    for cr in c_rels:
        left, right = cr.replace("Causer:","").replace("Result:","").split("->")
        if left not in un_tags or right not in un_tags:
            print("lr:",left,right)
            print("ix:",eix,six)
            print()
            continue
        l_ix = un_tags.index(left)
        r_ix = un_tags.index(right)
        if l_ix > r_ix:
            # swap so l_ix < r_ix
            r_ix, l_ix = l_ix, r_ix
        positions.append((l_ix, r_ix))
    
    non_proj = set()
    for (l,r) in positions:
        for (l1,r1) in positions:
            if l == l1 and r == r1:
                continue
            if (l1 > l and l1 < r) or (r1 > l and r1 < r):
                non_proj.add((eix, six))
    
non_proj

lr: 5 50
ix: 25 0

lr: 1 50
ix: 25 0

lr: 6 50
ix: 25 0



set()

In [20]:
tagged_essays[25].sentences[1]

[('some',
  {'Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('happen',
  {'Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('naturally',
  {'Causer:1->Result:50', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('like',
  {'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50',
   'explicit'}),
 ('shifting',
  {'1',
   'Causer',
   'Causer:1',
   'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50'}),
 ('trade',
  {'1',
   'Causer',
   'Causer:1',
   'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50'}),
 ('winds',
  {'1',
   'Causer',
   'Causer:1',
   'Causer:1->Result:50',
   'Causer:5->Result:50',
   'Causer:6->Result:50'}),
 ('and', {'Causer:5->Result:50', 'Causer:6->Result:50'}),
 ('photosynthesis',
  {'5', 'Causer', 'Causer:5', 'Causer:5->Result:50', 'Causer:6->Result:50'}),
 (',', set()),
 ('but', {'Causer:6->Result:50'}),
 ('others', {'Causer:6->Result:50'}),
 ('are',

## Are Causes Always < Relations? No....

In [22]:
for tag, freq in tag_freq.items():
    if "->" in tag:
        alt_tag = tag.replace("Causer:","").replace("Result:","").replace("b","")
        l,r = alt_tag.split("->")
        if l[0].isdigit() and r[0].isdigit():
            l,r = int(l), int(r)
            if r < l:
                print(tag.ljust(22), freq)
            #else:
            #    print("X", tag, freq)

Causer:3->Result:1     147
Causer:7->Result:5b    48
Causer:11->Result:3    46
Causer:11->Result:6    29
Causer:13->Result:11   20
Causer:11->Result:4    14
Causer:13->Result:12   28
Causer:14->Result:6    9
Causer:14->Result:7    44
Causer:13->Result:6    38
Causer:5->Result:3     13
Causer:50->Result:7    49
Causer:50->Result:1    7
Causer:12->Result:5b   9
Causer:12->Result:7    33
Causer:5->Result:4     11
Causer:7->Result:5     12
Causer:13->Result:4    11
Causer:13->Result:7    28
Causer:4->Result:3     21
Causer:13->Result:5    9
Causer:50->Result:3    9
Causer:7->Result:4     19
Causer:2->Result:1     10
Causer:3->Result:2     19
Causer:6->Result:5b    7
Causer:12->Result:11   13


## Note You Can Set Per Observation Weights in XGBoost
* http://xgboost.readthedocs.io/en/latest/python/python_api.html#module-xgboost.training
  * see weight parameter in dtrain